<a href="https://www.kaggle.com/code/tazegul20/ielts-prediction?scriptVersionId=157848594" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_excel(io="/kaggle/input/ielts-success-stories-dataset/IELTS_Success_Stories_Dataset.xlsx",sheet_name='Sheet1')
df.head()

In [ ]:
df. isnull().sum()

In [ ]:
df.drop("Profession",inplace=True,axis=1)

In [ ]:
df.head()

# Simple Regression Model 

This will be a very simple model for beginners who can use it as a template but at the next header I'll use more complex models to increase the accuracy score.



In [ ]:
X = df.drop("Location",axis=1)
y =df["IELTS Score (Overall)"]

First we need to prepare data to fit on model. We've got categorical and numerical variables so I'll get dummies for the categorical variables but at the next header I'll show you how can you do it by using Scikit-Learn.

![](http://cdn-coiao.nitrocdn.com/CYHudqJZsSxQpAPzLkHFOkuzFKDpEHGF/assets/images/optimized/rev-7841bc2/www.sharpsightlabs.com/wp-content/uploads/2022/04/pandas-getdummies_simple-visual-example.png)


In [ ]:
X.head()

Now Lets select our features based on their correlation coefficient.

![](https://qph.cf2.quoracdn.net/main-qimg-8b7cfb2dfcd459f25c9d7641aacc82ab-pjlq)

In [ ]:
X.head()

In [ ]:
X = pd.get_dummies(X, drop_first = True)
cor=X.corr()
cor_target = abs(cor["IELTS Score (Overall)"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features


These are our relevant features and we'll use these features as indepent variables

In [ ]:
X=X.loc[:,['Practice Hours per Week','Mock Tests Taken','Key Strategies_Self-study, vocabulary building, speaking practice','English Proficiency (Preparation)_Intermediate','Achieved Desired Score_Yes','Application Status_Under Process','Additional Certifications_CFA Level I','Volunteer Experience_Yes','Internship Experience_Financial Analyst','Relevant Skills_Financial Modeling, Data Analysis','Recommendations_Positive','Networking Efforts_Online Forums']]
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
mdl = LinearRegression()
mdl.fit(X_train, y_train)

ypred_train = mdl.predict(X_train)
ypred_test = mdl.predict(X_test)

print("Train: ", r2_score(y_train, ypred_train))
print("Test: ", r2_score(y_test, ypred_test))


Ok. We got 0.91 for train 0.87 for test What does it mean ? R squared is regression score function the best possible score you can get is 1.0 and worst is 0.0 so we got a good score but you need to aware of overfitting if there was huge difference between our train and test score for exampla 0.91 and 0.45 we would say that our model is **overfit**
![](https://www.veribilimiokulu.com/wp-content/uploads/2020/02/overfit.jpg.webp)


This is end of the simple model next header I'll show you how you can use scikit-learn for better models 

# Advanced Model

In [ ]:
df.head()

Clear the features which we don't need 

In [ ]:
df.drop(['Candidate','Location','Target Country','Job Offer Received','Application Status','Language Fluency','Age'],inplace=True,axis=1)

In [ ]:
df['Education Level'].value_counts()

In [ ]:
df['Recommendations'].value_counts()

In [ ]:
df['Preferred Learning Resources'].value_counts()

In [ ]:
df.rename(columns={'English Proficiency (Preparation)': 'English_Proficiency'},inplace=True,)

In [ ]:
df['Key Strategies'].value_counts()

We can see that some of the categorical variables need to be encoded by OneHotencoder and rest of them will be labeled by labelencoder

In [ ]:
label_encoded_cats = df.loc[:,['Education Level','English_Proficiency','Recommendations']]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(label_encoded_cats['Education Level'])
le.classes_
le.transform(label_encoded_cats['Education Level'])

In [ ]:
le.fit(label_encoded_cats['English_Proficiency'])
le.transform(label_encoded_cats['English_Proficiency'])

In [ ]:

le.fit(label_encoded_cats['Recommendations'])
le.transform(label_encoded_cats['Recommendations'])

In [ ]:
label_encoded_cats['English_Proficiency']=le.transform(label_encoded_cats['English_Proficiency'])

In [ ]:
label_encoded_cats['Recommendations']=le.transform(label_encoded_cats['Recommendations'])

In [ ]:
label_encoded_cats['Education Level']=le.transform(label_encoded_cats['Education Level'])

ValueError: y contains previously unseen labels: "Master's"

If you get that error please run
le.fit(label_encoded_cats['Education Level'])
le.classes_
le.transform(label_encoded_cats['Education Level'])

this line again and try one more time

# Labeled CATEGORIES

In [ ]:
label_encoded_cats

In [ ]:
df.head()

In [ ]:
df['Networking Efforts']

In [ ]:
one_hoted_cats=df.loc[:,['Profession','Key Strategies','Preferred Learning Resources','Additional Certifications','Volunteer Experience','Internship Experience','Relevant Skills','Networking Efforts']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore").set_output(transform = "pandas")
encoder.fit(one_hoted_cats)
one_hoted_cats=encoder.transform(one_hoted_cats)

In [ ]:
one_hoted_cats

In [ ]:
df=pd.concat([df.drop(['Profession','Key Strategies','Preferred Learning Resources','Additional Certifications','Volunteer Experience','Internship Experience','Relevant Skills','Networking Efforts'],axis=1),one_hoted_cats],axis=1)

In [ ]:
df.head()

In [ ]:
df.drop('Achieved Desired Score',axis=1,inplace=True)


In [ ]:
df= pd.concat([df.drop(['Education Level','English_Proficiency','Recommendations'],axis=1),label_encoded_cats],axis=1)

In [ ]:
df.head()

# Bingo! Our data is ready!

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop('IELTS Score (Overall)',axis=1)
y=df['IELTS Score (Overall)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

selector2=VarianceThreshold()
selector2.fit(X_train,y_train)
X_train = selector2.transform(X_train)
X_test = selector2.transform(X_test)

In [ ]:
import xgboost as xg 
xgb_r = xg.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123)
xgb_r.fit(X_train, y_train)   
pred = xgb_r.predict(X_test) 
print(r2_score(y_test, pred))

https://machinelearningmastery.com/xgboost-for-regression/
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold

Please check out the links if you don't understand the libraries I used in the model. By the way, you can use GridSearch for beter parameters and its link is the following:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

# Last Thing

* I'm still trying to learn these algorithms, so if I've forgotten anything, don't forget to let me know in the comments. I wish success to everyone :D